# KL By Removed

In [1]:
import numpy as np
from lib.utils import model_results_from_npz
import csv

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 100
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100
DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"
EPS = 1e-12

# --------------------
# Prepare CSV
# --------------------
csv_path = "../analytics/CIFAR100/kl/kl_per_removed.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Seed", "Removed_Class", "KL_Forget", "KL_Retain", "KL_Whole"])

    # --------------------
    # Loop over seeds & removed classes
    # --------------------
    for seed in SEEDS:
        for removed_class in REMOVED_CLASSES:

            # Load models
            retrain_path = f"{DIR_RETRAIN}/cifar100_resnet18_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # --------------------
            # KL on forget class
            # --------------------
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            # Normalize safely
            conf_retrain_forget /= (conf_retrain_forget.sum(axis=1, keepdims=True) + EPS)
            conf_redis_forget   /= (conf_redis_forget.sum(axis=1, keepdims=True) + EPS)
            kl_forget = np.sum(conf_retrain_forget * np.log((conf_retrain_forget + EPS) / (conf_redis_forget + EPS)), axis=1).mean()

            # --------------------
            # KL on retain classes
            # --------------------
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c]   for c in retain_classes])
            conf_retrain_retain /= (conf_retrain_retain.sum(axis=1, keepdims=True) + EPS)
            conf_redis_retain   /= (conf_redis_retain.sum(axis=1, keepdims=True) + EPS)
            kl_retain = np.sum(conf_retrain_retain * np.log((conf_retrain_retain + EPS) / (conf_redis_retain + EPS)), axis=1).mean()

            # --------------------
            # KL on whole dataset
            # --------------------
            conf_retrain_all = np.vstack([model_retrain.confidences[c] for c in range(NUM_CLASSES)])
            conf_redis_all   = np.vstack([model_redis.confidences[c]   for c in range(NUM_CLASSES)])
            conf_retrain_all /= (conf_retrain_all.sum(axis=1, keepdims=True) + EPS)
            conf_redis_all   /= (conf_redis_all.sum(axis=1, keepdims=True) + EPS)
            kl_whole = np.sum(conf_retrain_all * np.log((conf_retrain_all + EPS) / (conf_redis_all + EPS)), axis=1).mean()

            # Write row
            writer.writerow([seed, removed_class, kl_forget, kl_retain, kl_whole])
            print(f"Seed {seed}, Removed Class {removed_class} | KL Forget: {kl_forget:.6f}, KL Retain: {kl_retain:.6f}, KL Whole: {kl_whole:.6f}")

print(f"CSV saved to {csv_path}")


Seed 42, Removed Class 4 | KL Forget: 1.267872, KL Retain: 0.548356, KL Whole: 0.555551
Seed 42, Removed Class 7 | KL Forget: 1.265855, KL Retain: 0.548469, KL Whole: 0.555643
Seed 42, Removed Class 6 | KL Forget: 1.235624, KL Retain: 0.559730, KL Whole: 0.566489
Seed 42, Removed Class 11 | KL Forget: 0.955372, KL Retain: 0.546972, KL Whole: 0.551056
Seed 42, Removed Class 14 | KL Forget: 1.222185, KL Retain: 0.544035, KL Whole: 0.550816
Seed 42, Removed Class 15 | KL Forget: 1.592535, KL Retain: 0.559776, KL Whole: 0.570104
Seed 42, Removed Class 19 | KL Forget: 1.249975, KL Retain: 0.551267, KL Whole: 0.558254
Seed 42, Removed Class 24 | KL Forget: 0.775004, KL Retain: 0.555474, KL Whole: 0.557669
Seed 42, Removed Class 23 | KL Forget: 1.156062, KL Retain: 0.566663, KL Whole: 0.572557
Seed 42, Removed Class 25 | KL Forget: 0.956913, KL Retain: 0.557593, KL Whole: 0.561586
Seed 42, Removed Class 29 | KL Forget: 1.344301, KL Retain: 0.548816, KL Whole: 0.556771
Seed 42, Removed Class 3

# KL Per Seed

In [2]:
import numpy as np
from lib.utils import model_results_from_npz
import csv

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 100
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100
DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"
EPS = 1e-12

# --------------------
# Prepare CSV
# --------------------
csv_path = "../analytics/CIFAR100/kl/kl_summary_per_seed.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Seed", "KL_Forget_avg", "KL_Retain_avg", "KL_Whole_avg"])

    # --------------------
    # Loop over seeds
    # --------------------
    for seed in SEEDS:
        kl_forget_list, kl_retain_list, kl_whole_list = [], [], []

        for removed_class in REMOVED_CLASSES:
            # Load models
            retrain_path = f"{DIR_RETRAIN}/cifar100_resnet18_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # KL on forget class
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            conf_retrain_forget /= (conf_retrain_forget.sum(axis=1, keepdims=True) + EPS)
            conf_redis_forget   /= (conf_redis_forget.sum(axis=1, keepdims=True) + EPS)
            kl_forget_list.append(np.sum(conf_retrain_forget * np.log((conf_retrain_forget + EPS) / (conf_redis_forget + EPS)), axis=1).mean())

            # KL on retain classes
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c]   for c in retain_classes])
            conf_retrain_retain /= (conf_retrain_retain.sum(axis=1, keepdims=True) + EPS)
            conf_redis_retain   /= (conf_redis_retain.sum(axis=1, keepdims=True) + EPS)
            kl_retain_list.append(np.sum(conf_retrain_retain * np.log((conf_retrain_retain + EPS) / (conf_redis_retain + EPS)), axis=1).mean())

            # KL on whole dataset
            conf_retrain_all = np.vstack([model_retrain.confidences[c] for c in range(NUM_CLASSES)])
            conf_redis_all   = np.vstack([model_redis.confidences[c]   for c in range(NUM_CLASSES)])
            conf_retrain_all /= (conf_retrain_all.sum(axis=1, keepdims=True) + EPS)
            conf_redis_all   /= (conf_redis_all.sum(axis=1, keepdims=True) + EPS)
            kl_whole_list.append(np.sum(conf_retrain_all * np.log((conf_retrain_all + EPS) / (conf_redis_all + EPS)), axis=1).mean())

        # Average over all removed classes for this seed
        kl_forget_avg = np.mean(kl_forget_list)
        kl_retain_avg = np.mean(kl_retain_list)
        kl_whole_avg  = np.mean(kl_whole_list)

        writer.writerow([seed, kl_forget_avg, kl_retain_avg, kl_whole_avg])
        print(f"Seed {seed} | KL Forget Avg: {kl_forget_avg:.6f}, KL Retain Avg: {kl_retain_avg:.6f}, KL Whole Avg: {kl_whole_avg:.6f}")

print(f"CSV saved to {csv_path}")


Seed 42 | KL Forget Avg: 1.163914, KL Retain Avg: 0.559588, KL Whole Avg: 0.565631
Seed 602 | KL Forget Avg: 1.159101, KL Retain Avg: 0.563368, KL Whole Avg: 0.569325
Seed 311 | KL Forget Avg: 1.135363, KL Retain Avg: 0.558440, KL Whole Avg: 0.564209
Seed 637 | KL Forget Avg: 1.183099, KL Retain Avg: 0.567969, KL Whole Avg: 0.574120
Seed 800 | KL Forget Avg: 1.167035, KL Retain Avg: 0.557740, KL Whole Avg: 0.563833
Seed 543 | KL Forget Avg: 1.166650, KL Retain Avg: 0.558220, KL Whole Avg: 0.564304
Seed 969 | KL Forget Avg: 1.172708, KL Retain Avg: 0.555213, KL Whole Avg: 0.561388
Seed 122 | KL Forget Avg: 1.149785, KL Retain Avg: 0.554743, KL Whole Avg: 0.560693
Seed 336 | KL Forget Avg: 1.168998, KL Retain Avg: 0.565023, KL Whole Avg: 0.571063
Seed 93 | KL Forget Avg: 1.119879, KL Retain Avg: 0.552632, KL Whole Avg: 0.558304
CSV saved to ../analytics/CIFAR100/kl/kl_summary_per_seed.csv


# KL Three-Way

In [3]:

# %%
import os
import numpy as np
import csv
from lib.utils import model_results_from_npz

# --------------------
# Config
# --------------------
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
NUM_CLASSES = 100
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100
DIR_ORIG   = "../results/cifar100/"
DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"
EPS = 1e-12

# --------------------
# Output CSV
# --------------------
csv_path = "../analytics/CIFAR100/kl/kl_original_retrain_redis.csv"
os.makedirs(os.path.dirname(csv_path), exist_ok=True)

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Seed", "Removed_Class",
        "KL_Orig_Redis_Forget", "KL_Orig_Redis_Retain",
        "KL_Orig_Retrain_Forget", "KL_Orig_Retrain_Retain",
        "KL_Redis_Retrain_Forget", "KL_Redis_Retrain_Retain"
    ])

    # --------------------
    # Loop over seeds & removed classes
    # --------------------
    for seed in SEEDS:
        print(f"\n=== Seed {seed} ===")
        for removed_class in REMOVED_CLASSES:
            # Paths
            orig_path    = f"{DIR_ORIG}/cifar100_resnet18_s{seed}_e{EPOCHS}.npz"
            retrain_path = f"{DIR_RETRAIN}/cifar100_resnet18_s{seed}_e{EPOCHS}_r{removed_class}.npz"
            redis_path   = f"{DIR_REDIS}/cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz"

            if not (os.path.exists(orig_path) and os.path.exists(retrain_path) and os.path.exists(redis_path)):
                print(f"  Skipping class {removed_class} — missing file.")
                continue

            # Load models
            model_orig    = model_results_from_npz(orig_path, NUM_CLASSES)
            model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
            model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

            # --------------------
            # Helper function
            # --------------------
            def kl_between(conf_a, conf_b):
                conf_a = conf_a / (conf_a.sum(axis=1, keepdims=True) + EPS)
                conf_b = conf_b / (conf_b.sum(axis=1, keepdims=True) + EPS)
                return np.sum(conf_a * np.log((conf_a + EPS) / (conf_b + EPS)), axis=1).mean()

            # --------------------
            # Split into forget / retain subsets
            # --------------------
            retain_classes = [c for c in range(NUM_CLASSES) if c != removed_class]
            conf_orig_forget    = model_orig.confidences[removed_class]
            conf_retrain_forget = model_retrain.confidences[removed_class]
            conf_redis_forget   = model_redis.confidences[removed_class]
            conf_orig_retain    = np.vstack([model_orig.confidences[c] for c in retain_classes])
            conf_retrain_retain = np.vstack([model_retrain.confidences[c] for c in retain_classes])
            conf_redis_retain   = np.vstack([model_redis.confidences[c] for c in retain_classes])

            # --------------------
            # Compute all pairwise KL
            # --------------------
            kl_orig_redis_forget   = kl_between(conf_orig_forget,   conf_redis_forget)
            kl_orig_redis_retain   = kl_between(conf_orig_retain,   conf_redis_retain)
            kl_orig_retrain_forget = kl_between(conf_orig_forget,   conf_retrain_forget)
            kl_orig_retrain_retain = kl_between(conf_orig_retain,   conf_retrain_retain)
            kl_redis_retrain_forget = kl_between(conf_retrain_forget, conf_redis_forget)
            kl_redis_retrain_retain = kl_between(conf_retrain_retain, conf_redis_retain)

            # --------------------
            # Write to CSV
            # --------------------
            writer.writerow([
                seed, removed_class,
                kl_orig_redis_forget, kl_orig_redis_retain,
                kl_orig_retrain_forget, kl_orig_retrain_retain,
                kl_redis_retrain_forget, kl_redis_retrain_retain
            ])

            print(
                f"  Class {removed_class}: "
                f"O-Rd(F)={kl_orig_redis_forget:.3f}, O-Rd(R)={kl_orig_redis_retain:.3f} | "
                f"O-Rt(F)={kl_orig_retrain_forget:.3f}, O-Rt(R)={kl_orig_retrain_retain:.3f} | "
                f"Rd-Rt(F)={kl_redis_retrain_forget:.3f}, Rd-Rt(R)={kl_redis_retrain_retain:.3f}"
            )

print(f"\nCSV saved to {csv_path}")



=== Seed 42 ===
  Class 4: O-Rd(F)=15.246, O-Rd(R)=0.130 | O-Rt(F)=7.363, O-Rt(R)=0.609 | Rd-Rt(F)=1.268, Rd-Rt(R)=0.548
  Class 7: O-Rd(F)=19.499, O-Rd(R)=0.075 | O-Rt(F)=9.118, O-Rt(R)=0.582 | Rd-Rt(F)=1.266, Rd-Rt(R)=0.548
  Class 6: O-Rd(F)=21.079, O-Rd(R)=0.081 | O-Rt(F)=9.518, O-Rt(R)=0.592 | Rd-Rt(F)=1.236, Rd-Rt(R)=0.560
  Class 11: O-Rd(F)=12.908, O-Rd(R)=0.142 | O-Rt(F)=7.142, O-Rt(R)=0.621 | Rd-Rt(F)=0.955, Rd-Rt(R)=0.547
  Class 14: O-Rd(F)=16.543, O-Rd(R)=0.093 | O-Rt(F)=6.918, O-Rt(R)=0.588 | Rd-Rt(F)=1.222, Rd-Rt(R)=0.544
  Class 15: O-Rd(F)=20.134, O-Rd(R)=0.080 | O-Rt(F)=8.751, O-Rt(R)=0.597 | Rd-Rt(F)=1.593, Rd-Rt(R)=0.560
  Class 19: O-Rd(F)=17.756, O-Rd(R)=0.084 | O-Rt(F)=7.474, O-Rt(R)=0.581 | Rd-Rt(F)=1.250, Rd-Rt(R)=0.551
  Class 24: O-Rd(F)=21.859, O-Rd(R)=0.036 | O-Rt(F)=11.057, O-Rt(R)=0.574 | Rd-Rt(F)=0.775, Rd-Rt(R)=0.555
  Class 23: O-Rd(F)=22.578, O-Rd(R)=0.047 | O-Rt(F)=9.960, O-Rt(R)=0.593 | Rd-Rt(F)=1.156, Rd-Rt(R)=0.567
  Class 25: O-Rd(F)=14.876, O-R